In [33]:
import re
import os
import pathlib
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import gridspec
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
sns.set_theme(style="darkgrid")

# Root of the project
project_dir = pathlib.Path().resolve().parent

# Directory containing experiment results
results_dir = (pathlib.Path(project_dir) / "results").resolve()

# Directory for tidied data (output directory)
data_dir = (pathlib.Path(project_dir) / "data").resolve()

# Directory to store all plots
plot_dir = (pathlib.Path(project_dir) / "plots").resolve()



# Experiment constants
exp_name = "02_http_constant_throughput"
file_prefix = "exp-02"

In [34]:

"""Load cleaned/aggregated data sets

resource_df -> CPU/Mem utilization for experiments
fortio_df -> Load test results from fortio
"""

resource_df = pd.read_feather(data_dir / "resource_results.feather")
fortio_df = pd.read_feather(data_dir / "fortio_results.feather")

In [35]:
"""
Prepares the experiment data

"""


exp = fortio_df[fortio_df["experiment"] == exp_name]
exp = exp.drop("actual_qps", 1)
exp = exp.explode("latency")
exp = exp.reset_index(drop=True)
exp = exp.astype({
    "latency": np.float64
})

exp = exp.sort_values("mesh", ascending=True)


exp.head()

,experiment,mesh,requested_qps,protocol,payload,latency
6601700,02_http_constant_throughput,Baseline,100,http,0,110.851703
4953938,02_http_constant_throughput,Baseline,1000,http,0,0.123170
4953939,02_http_constant_throughput,Baseline,1000,http,0,0.115875
4953940,02_http_constant_throughput,Baseline,1000,http,0,0.197469
4953941,02_http_constant_throughput,Baseline,1000,http,0,0.147604
